# Login to https://community.pipeline.ai

# Deploy Model to PipelineAI

You will need to fill in the unique values for the following:
* `<YOUR_USER_ID>`  - 8 character id that uniquely identifies the PipelineAI user.  You will see the UserId in the upper right hand corner of the Settings tab after you login to [PipelineAI Community Edition](https://community.cloud.pipeline.ai)
* `<UNIQUE_MODEL_NAME>` - User-defined model/project name that uniquely identifies a model/project within your account.
* `<UNIQUE_TAG_NAME>` - User-defined tag that uniquely identifies the model tag/version within a model/project


![user-id](https://pipeline.ai/assets/img/user-id.png)

```
pipeline resource-upload --host community.cloud.pipeline.ai --user-id <YOUR_USER_ID> --resource-type model --resource-subtype tensorflow  --name <UNIQUE_MODEL_NAME> --tag <UNIQUE_TAG_NAME> --path ./tensorflow/mnist-v1/model
```

Actions performed:
* Compress resource source code into a tar archive.
* Create required directories and generate deployment and service resource definitions.
* Receive resource source code - or trained binary (ie. tensorflow SavedModel binary)
  from client as a tar archive then uncompress and extract on the PipelineAI server.
* Initialize training resource

# Optimize and Deploy the Model
You can optimize (select one or more chips and/or one or more runtimes) and deploy your model using the CLI or the [UI](https://community.cloud.pipeline.ai) (Choose either the CLI or UI).


In [ ]:
import numpy as np
import pylab
from tensorflow.contrib.kafka.python.ops import kafka_dataset_ops
from tensorflow.python.data.ops import iterator_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import errors
from tensorflow.python.ops import array_ops
from tensorflow.python.platform import test
import tensorflow as tf
from confluent_kafka import Producer
import json


num_samples = 100

x_train = np.random.rand(num_samples).astype(np.float32)
print(x_train)

noise = np.random.normal(scale=0.01, size=len(x_train))

y_train = x_train * 0.1 + 0.3 + noise
print(y_train)

pylab.plot(x_train, y_train, '.')

In [ ]:

x_test = np.random.rand(len(x_train)).astype(np.float32)
print(x_test)

noise = np.random.normal(scale=.01, size=len(x_train))

y_test = x_test * 0.1 + 0.3 + noise
print(y_test)

pylab.plot(x_test, y_test, '.')

In [ ]:
topic_name = '83f05e58' + 'linear'

p = Producer({'bootstrap.servers': 'kafka-cp-kafka-headless:9092'})

for x, y in zip(x_train, y_train):
    p.produce(topic_name, str((x, y)).encode('utf-8'))

if (p.flush() == 0):
    print('Success')
else:
    print('Failure')

In [ ]:
def getSingleRecord(topic_name):
    topics = array_ops.placeholder(dtypes.string, shape=[None])
    num_epochs = array_ops.placeholder(dtypes.int64, shape=[])
    batch_size = array_ops.placeholder(dtypes.int64, shape=[])

    repeat_dataset = kafka_dataset_ops.KafkaDataset(
        servers="kafka-cp-kafka-headless:9092", 
        topics=[topic_name], 
        group="community", 
        eof=False).repeat(num_epochs)
    
    batch_dataset = repeat_dataset.batch(batch_size)

    iterator = iterator_ops.Iterator.from_structure(batch_dataset.output_types)
    init_op = iterator.make_initializer(repeat_dataset)
    get_next = iterator.get_next()
    
    with tf.Session() as sess:
        # Read from topic
        sess.run(
            init_op,
            feed_dict={
              topics: [topic_name],
              num_epochs: 1,
              batch_size: 5
            })
        try:
            print(sess.run(get_next).decode('utf-8'))
        except:
            print('No more messages to process.')


In [ ]:
record = getSingleRecord(topic_name)

In [ ]:
def getMultipleRecords(topic_name):
    topics = array_ops.placeholder(dtypes.string, shape=[None])
    num_epochs = array_ops.placeholder(dtypes.int64, shape=[])
    batch_size = array_ops.placeholder(dtypes.int64, shape=[])

    repeat_dataset = kafka_dataset_ops.KafkaDataset(
      servers="kafka-cp-kafka-headless:9092", 
      topics=[topic_name], 
      group="community", 
      eof=False).repeat(num_epochs)

    batch_dataset = repeat_dataset.batch(batch_size)

    iterator = iterator_ops.Iterator.from_structure(batch_dataset.output_types)
    init_op = iterator.make_initializer(repeat_dataset)
    get_next = iterator.get_next()

    with tf.Session() as sess:
        # Batched and repeated iteration through the stream.
        init_batch_op = iterator.make_initializer(batch_dataset)
        sess.run(
          init_batch_op,
            feed_dict={
            topics: [topic_name],
            num_epochs: 20,
            batch_size: 5
        })

        more_records = True
        while more_records:
           try:
               print(sess.run(get_next))
           except e:
               print('No more messages to process.')
               more_records = False


In [ ]:
records = getMultipleRecords(topic_name)